# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import math
from math import floor
# data: https://www.kaggle.com/c/nfl-big-data-bowl-2021/data

Source: https://www.nfeloapp.com/analysis/expected-points-added-epa-nfl

In [2]:
first_down_slope = [-.75, 5]
second_down_slope = [-1, 4.5]
third_down_slope = [-2, 3.75]
fourth_down_slope = [-3, 2.5]
turnover_down_slope = [-5, .75]

EPA slopes estimated from NFEloApp chart

In [3]:
first_down_est = np.linspace(first_down_slope[0], first_down_slope[1], 100)
second_down_est = np.linspace(second_down_slope[0], second_down_slope[1], 100)
third_down_est = np.linspace(third_down_slope[0], third_down_slope[1], 100)
fourth_down_est = np.linspace(fourth_down_slope[0], fourth_down_slope[1], 100)
turnover_down_est = np.linspace(turnover_down_slope[0], turnover_down_slope[1], 100)

Convert to data frame

In [4]:
epa_chart = pd.DataFrame(np.round(np.array([first_down_est, second_down_est, third_down_est, fourth_down_est, turnover_down_est]), 2), index=[1, 2, 3, 4, 5], columns=list(range(100, 0, -1)))

In [5]:
epa_chart

,100,99,98,97,96,95,94,93,92,91,...,10,9,8,7,6,5,4,3,2,1
1,-0.75,-0.69,-0.63,-0.58,-0.52,-0.46,-0.40,-0.34,-0.29,-0.23,...,4.48,4.54,4.59,4.65,4.71,4.77,4.83,4.88,4.94,5.00
2,-1.00,-0.94,-0.89,-0.83,-0.78,-0.72,-0.67,-0.61,-0.56,-0.50,...,4.00,4.06,4.11,4.17,4.22,4.28,4.33,4.39,4.44,4.50
3,-2.00,-1.94,-1.88,-1.83,-1.77,-1.71,-1.65,-1.59,-1.54,-1.48,...,3.23,3.29,3.34,3.40,3.46,3.52,3.58,3.63,3.69,3.75
4,-3.00,-2.94,-2.89,-2.83,-2.78,-2.72,-2.67,-2.61,-2.56,-2.50,...,2.00,2.06,2.11,2.17,2.22,2.28,2.33,2.39,2.44,2.50
5,-5.00,-4.94,-4.88,-4.83,-4.77,-4.71,-4.65,-4.59,-4.54,-4.48,...,0.23,0.29,0.34,0.40,0.46,0.52,0.58,0.63,0.69,0.75


Load data on plays and games

In [6]:
plays = pd.read_csv('./nfl-big-data-bowl-2021/plays.csv')
plays.head(2)

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,...,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI
0,2018090600,75,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,ATL,play_type_pass,ATL,20,...,0.0,15:00:00,90.0,NaN,NaN,C,10,10,0.261827,False
1,2018090600,146,(13:10) M.Ryan pass incomplete short right to ...,1,1,10,ATL,play_type_pass,PHI,39,...,0.0,13:10:00,49.0,NaN,NaN,I,0,0,-0.372360,False


In [7]:
games = pd.read_csv('./nfl-big-data-bowl-2021/games.csv')
games.head(2)

,gameId,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr
0,2018090600,9/6/18,20:20:00,PHI,ATL
1,2018090901,9/9/18,13:00:00,CLE,PIT


Import data stores

In [8]:
path = './data/stores/targeted.pkl'
with open(path, 'rb') as f:
    targeted = pickle.load(f)

In [9]:
path = './models/ens_predictions.pkl'
with open(path, 'rb') as f:
    preds = pickle.load(f)

# Data Merge

In [10]:
preds.head()

,gameId,playId,frameId,event,displayName,position,x,y,preds,team,pass_complete
0,2018090600,75,11,ball_snap,Matt Ryan,QB,91.74,26.67,0.457552,away,1
1,2018090600,75,11,ball_snap,Julio Jones,WR,91.36,44.14,0.578130,away,1
2,2018090600,75,11,ball_snap,Mohamed Sanu,WR,90.78,36.16,0.474055,away,1
3,2018090600,75,11,ball_snap,Devonta Freeman,RB,98.25,26.62,0.474567,away,1
4,2018090600,75,11,ball_snap,Austin Hooper,TE,91.30,21.85,0.540093,away,1


Add team abbreviation to preds frame

In [11]:
preds['team_abr'] = ''
all_games = preds['gameId'].unique()
for i, game in enumerate(all_games):
    for team in ['home', 'away']:
        
        if team == 'away':
            abr = games.loc[(games['gameId'] == game), 'visitorTeamAbbr'].iloc[0]
            preds.loc[(preds['gameId'] == game) & (preds['team'] == team), 'team_abr'] = abr

        if team == 'home':
            abr = games.loc[(games['gameId'] == game), 'homeTeamAbbr'].iloc[0]
            preds.loc[(preds['gameId'] == game) & (preds['team'] == team), 'team_abr'] = abr
    print(f'{i+1}/{len(all_games)} complete')

1/253 complete
2/253 complete
3/253 complete
4/253 complete
5/253 complete
6/253 complete
7/253 complete
8/253 complete
9/253 complete
10/253 complete
11/253 complete
12/253 complete
13/253 complete
14/253 complete
15/253 complete
16/253 complete
17/253 complete
18/253 complete
19/253 complete
20/253 complete
21/253 complete
22/253 complete
23/253 complete
24/253 complete
25/253 complete
26/253 complete
27/253 complete
28/253 complete
29/253 complete
30/253 complete
31/253 complete
32/253 complete
33/253 complete
34/253 complete
35/253 complete
36/253 complete
37/253 complete
38/253 complete
39/253 complete
40/253 complete
41/253 complete
42/253 complete
43/253 complete
44/253 complete
45/253 complete
46/253 complete
47/253 complete
48/253 complete
49/253 complete
50/253 complete
51/253 complete
52/253 complete
53/253 complete
54/253 complete
55/253 complete
56/253 complete
57/253 complete
58/253 complete
59/253 complete
60/253 complete
61/253 complete
62/253 complete
63/253 complete
6

In [12]:
preds.head()

,gameId,playId,frameId,event,displayName,position,x,y,preds,team,pass_complete,team_abr
0,2018090600,75,11,ball_snap,Matt Ryan,QB,91.74,26.67,0.457552,away,1,ATL
1,2018090600,75,11,ball_snap,Julio Jones,WR,91.36,44.14,0.578130,away,1,ATL
2,2018090600,75,11,ball_snap,Mohamed Sanu,WR,90.78,36.16,0.474055,away,1,ATL
3,2018090600,75,11,ball_snap,Devonta Freeman,RB,98.25,26.62,0.474567,away,1,ATL
4,2018090600,75,11,ball_snap,Austin Hooper,TE,91.30,21.85,0.540093,away,1,ATL


In [13]:
# Initialize columns
preds['epa_actual'] = 0 #
preds['abs_yardline'] = 0 #
preds['down'] = 0 #
preds['yds_to_go'] = 10 #
preds['play_result'] = 0 #

# Merge in play & game state data
all_games = preds['gameId'].unique()
for i, game in enumerate(all_games):
    all_plays = preds[(preds['gameId'] == game)]['playId'].unique()
    
    for play in all_plays:
        epa = plays.loc[(plays['gameId'] == game) & (plays['playId'] == play), 'epa'].iloc[0]
        preds.loc[(preds['gameId'] == game) & (preds['playId'] == play), 'epa_actual'] = epa

        # abs_yardline
        abs_yd = plays.loc[(plays['gameId'] == game) & (plays['playId'] == play), 'absoluteYardlineNumber'].iloc[0] - 10    # adjust yardline to absolute yardline. Original data includes end zone
        preds.loc[(preds['gameId'] == game) & (preds['playId'] == play), 'abs_yardline'] = abs_yd

        # down
        down = plays.loc[(plays['gameId'] == game) & (plays['playId'] == play), 'down'].iloc[0]                             # merge in down
        preds.loc[(preds['gameId'] == game) & (preds['playId'] == play), 'down'] = down
        
        # yardsToGo
        yds_to_go = plays.loc[(plays['gameId'] == game) & (plays['playId'] == play), 'yardsToGo'].iloc[0]                   # merge in distance
        preds.loc[(preds['gameId'] == game) & (preds['playId'] == play), 'yds_to_go'] = yds_to_go
        
        # offensive Play Result
        play_result = plays.loc[(plays['gameId'] == game) & (plays['playId'] == play), 'offensePlayResult'].iloc[0]         # get play result
        preds.loc[(preds['gameId'] == game) & (preds['playId'] == play), 'play_result'] = play_result

    print(f'{i+1}/{len(all_games)} complete')

1/253 complete
2/253 complete
3/253 complete
4/253 complete
5/253 complete
6/253 complete
7/253 complete
8/253 complete
9/253 complete
10/253 complete
11/253 complete
12/253 complete
13/253 complete
14/253 complete
15/253 complete
16/253 complete
17/253 complete
18/253 complete
19/253 complete
20/253 complete
21/253 complete
22/253 complete
23/253 complete
24/253 complete
25/253 complete
26/253 complete
27/253 complete
28/253 complete
29/253 complete
30/253 complete
31/253 complete
32/253 complete
33/253 complete
34/253 complete
35/253 complete
36/253 complete
37/253 complete
38/253 complete
39/253 complete
40/253 complete
41/253 complete
42/253 complete
43/253 complete
44/253 complete
45/253 complete
46/253 complete
47/253 complete
48/253 complete
49/253 complete
50/253 complete
51/253 complete
52/253 complete
53/253 complete
54/253 complete
55/253 complete
56/253 complete
57/253 complete
58/253 complete
59/253 complete
60/253 complete
61/253 complete
62/253 complete
63/253 complete
6

In [14]:
preds.iloc[200:20,:]

,gameId,playId,frameId,event,displayName,position,x,y,preds,team,pass_complete,team_abr,epa_actual,abs_yardline,down,yds_to_go,play_result


Estimate yard and down of next play if pass is complete at every frame of the current play

In [15]:
# impute missing values
_ = np.where(preds['abs_yardline'].isna())
preds.loc[_[0], 'abs_yardline'] = preds.loc[_[0], 'x']

preds['est_yard_gained'] = (preds['x'] - 10)
preds['est_yard_gained'] = preds['abs_yardline'] - preds['est_yard_gained']
preds.iloc[175:190,:]

,gameId,playId,frameId,event,displayName,position,x,y,preds,team,pass_complete,team_abr,epa_actual,abs_yardline,down,yds_to_go,play_result,est_yard_gained
175,2018090600,75,40,None,Julio Jones,WR,80.19,45.67,0.697008,away,1,ATL,0.261827,80.0,1,15,10,9.81
176,2018090600,75,40,None,Mohamed Sanu,WR,80.30,34.73,0.711775,away,1,ATL,0.261827,80.0,1,15,10,9.70
177,2018090600,75,40,None,Devonta Freeman,RB,93.11,28.73,0.392266,away,1,ATL,0.261827,80.0,1,15,10,-3.11
178,2018090600,75,40,None,Austin Hooper,TE,80.88,20.46,0.634111,away,1,ATL,0.261827,80.0,1,15,10,9.12
179,2018090600,75,40,None,Ricky Ortiz,FB,88.49,15.79,0.678308,away,1,ATL,0.261827,80.0,1,15,10,1.51
180,2018090600,75,41,None,Matt Ryan,QB,96.24,27.14,0.451941,away,1,ATL,0.261827,80.0,1,15,10,-6.24
181,2018090600,75,41,None,Julio Jones,WR,80.23,45.92,0.692548,away,1,ATL,0.261827,80.0,1,15,10,9.77
182,2018090600,75,41,None,Mohamed Sanu,WR,79.81,34.79,0.725837,away,1,ATL,0.261827,80.0,1,15,10,10.19
183,2018090600,75,41,None,Devonta Freeman,RB,92.90,29.07,0.397534,away,1,ATL,0.261827,80.0,1,15,10,-2.90
184,2018090600,75,41,None,Austin Hooper,TE,80.38,20.08,0.624022,away,1,ATL,0.261827,80.0,1,15,10,9.62


Add in the down of the following play for EPA estimations

In [16]:
def next_down(x):
    """Estimates down of following play"""
    if x['est_next_down_yds'] >= 0:
        return 1
    else:
        next_down = x['down'] + 1
        return next_down

Add to preds frame

In [17]:
preds['est_next_down_yds'] = preds['est_yard_gained'] - preds['yds_to_go']
preds['est_next_down'] = preds.apply(next_down, axis=1)

Estimate expected points

In [18]:
def est_ep(x):
    """Estimates the expected points at a given yardline and down"""
    next_d = x['est_next_down'] 
    next_ydl = math.floor(x['abs_yardline'] - x['est_yard_gained'])
    if next_ydl < 1:
        return 6.95    # this is a touchdown + extra point
    if next_ydl > 100:
        next_d = 5
        next_ydl = 20
        return epa_chart.loc[next_d, next_ydl] + 2  # this is a safety
    return epa_chart.loc[next_d, next_ydl]

Estimate the expected points added

In [19]:
def est_epa(x):
    """Estimates the expected points added at every frame of every play"""
    down = x['down']
    ydl = int(math.floor(x['abs_yardline']))
    if ydl < 1:
        ydl = 1
    if ydl > 100:
        ydl = 100
        
    ep_after = x['est_ep']
    start_ep = epa_chart.loc[down, ydl]

    epa = ep_after - start_ep
    epa *= x['preds']

    return epa

Add EP to preds frame

In [20]:
preds['est_ep'] = preds.apply(est_ep, axis=1)

Add EPA to preds frame

In [21]:
preds['est_epa'] = preds.apply(est_epa, axis=1)

Multiple the estimated EPA at every frame of every play against the predicted catch percentage to estimate the expected EPA

In [22]:
preds['epa_prob'] = preds['est_epa'] * preds['preds']

Initialize target estimated epa, optimal epa on a given play, and the optimal receiver on a given play.

In [23]:
preds['target_est_epa'] = -50
preds['optimal_epa'] = -50
preds['optimal_receiver'] = 'None'

In [24]:
all_games = preds['gameId'].unique()

for i, game in enumerate(all_games):
    plays = preds[preds['gameId'] == game]['playId'].unique()

    for play in plays:
        
        if preds.loc[(preds['gameId'] == game) & (preds['playId'] == play)]['optimal_receiver'].iloc[0] == 'None':
    
            play_df = preds[(preds['gameId'] == game) & (preds['playId'] == play)].copy()

            if play_df[play_df['event'] == 'pass_forward'].shape[0] > 0:
                
                frameId = play_df[play_df['event'] == 'pass_forward']['frameId'].iloc[0]
                play_df = play_df.loc[play_df['frameId'] <= frameId]                                                                                              # get shortened frame
                
                optimal_epa_idx = play_df['epa_prob'].idxmax()
                optimal_epa = play_df.loc[optimal_epa_idx]['epa_prob']
                optimal_epa_rec = play_df.loc[optimal_epa_idx]['displayName']
                preds.loc[(preds['gameId'] == game) & (preds['playId'] == play), 'optimal_epa'] = optimal_epa
                preds.loc[(preds['gameId'] == game) & (preds['playId'] == play), 'optimal_receiver'] = optimal_epa_rec

                targeted_receiver = targeted[game][play]                                                                                                # get targeted receiver
                target_est_epa = play_df.loc[(play_df['displayName'] == targeted_receiver) & (play_df['frameId']==frameId)]['epa_prob'].iloc[0]         # get targets estimated epa
                preds.loc[(preds['gameId'] == game) & (preds['playId'] == play), 'target_est_epa'] = target_est_epa
            else:
                print(f'{game} -- {play} has no pass forward')

    if i % 5 == 0:
        print(f'{i+1}/{len(all_games)} games complete')


1/253 games complete
6/253 games complete
11/253 games complete
16/253 games complete
21/253 games complete
26/253 games complete
31/253 games complete
2018092304 -- 1687 has no pass forward
36/253 games complete
2018092301 -- 453 has no pass forward
2018092301 -- 477 has no pass forward
2018092301 -- 520 has no pass forward
2018092301 -- 1161 has no pass forward
2018092301 -- 1183 has no pass forward
2018092301 -- 1507 has no pass forward
2018092301 -- 1553 has no pass forward
2018092301 -- 1804 has no pass forward
2018092301 -- 1885 has no pass forward
2018092301 -- 2225 has no pass forward
2018092301 -- 2451 has no pass forward
2018092301 -- 2473 has no pass forward
2018092301 -- 2962 has no pass forward
2018092301 -- 2986 has no pass forward
2018092301 -- 3067 has no pass forward
2018092301 -- 3091 has no pass forward
2018092301 -- 3186 has no pass forward
2018092301 -- 3210 has no pass forward
2018092301 -- 3469 has no pass forward
2018092301 -- 3538 has no pass forward
2018092301

In [ ]:
path = './data/stores/wip_preds_file.pkl'
with open(path, 'wb') as f:
    pickle.dump(preds, f)

Split Data into individual game files

In [27]:
# read data
def load_data():
    weeks = list(range(1, 18))
    for week in weeks:
        path=f'./data/subset/web/week_{week}.csv'
        data = pd.read_csv(path, compression='zip')

        yield data

Rebuild play by play data frames for web with predictions and optimal epa estimates. Then reduce the size to the minimum number of columns needed for graphing. This will increase the graph building speed when displaying on the user's screen.

In [28]:
weeks = load_data()
# minimum number of columns from the preds frame needed
preds_small = preds[['gameId', 'playId', 'frameId', 'displayName', 'est_epa', 'preds', 'optimal_receiver', 'epa_actual', 'optimal_epa', 'down']]

for week in weeks:

    # minimum number of columns needed from the weeks frame, processed previously
    week_small = week[['week', 'gameId', 'playId', 'frameId', 'team_name', 'team', 'possession', 'displayName', 'position', 'x', 'y', 'event']]
    all_games = week_small['gameId'].unique()

    week_num = week_small['week'].iloc[0]
    output = pd.DataFrame()

    for i, game in enumerate(all_games):
        all_plays = week_small[(week_small['gameId'] == game)]['playId'].unique()

        for play in all_plays:

            play_out = week_small[(week_small['gameId'] == game) & (week_small['playId'] == play)].copy()

            if len(play_out[(play_out['gameId'] == game) & (play_out['playId'] == play)]['frameId'].unique()) > 15:

                    if 'pass_forward' in play_out[(play_out['gameId'] == game) & (play_out['playId'] == play)]['event'].unique():

                        merged_data = pd.merge(left=preds_small, right=play_out, left_on=['gameId', 'playId', 'frameId', 'displayName'], right_on=['gameId', 'playId', 'frameId', 'displayName'],
                        how='right')
                        output = pd.concat([output, merged_data], ignore_index=True)
                    
        print(f'{i+1}/{len(all_games)} completed')

            
    output.to_csv(f'./data/export/week_{week_num}.csv', index=False, compression='zip')     # export for web app


1/13 completed
2/13 completed
3/13 completed
4/13 completed
5/13 completed
6/13 completed
7/13 completed
8/13 completed
9/13 completed
10/13 completed
11/13 completed
12/13 completed
13/13 completed
1/16 completed
2/16 completed
3/16 completed
4/16 completed
5/16 completed
6/16 completed
7/16 completed
8/16 completed
9/16 completed
10/16 completed
11/16 completed
12/16 completed
13/16 completed
14/16 completed
15/16 completed
16/16 completed
1/16 completed
2/16 completed
3/16 completed
4/16 completed
5/16 completed
6/16 completed
7/16 completed
8/16 completed
9/16 completed
10/16 completed
11/16 completed
12/16 completed
13/16 completed
14/16 completed
15/16 completed
16/16 completed
1/15 completed
2/15 completed
3/15 completed
4/15 completed
5/15 completed
6/15 completed
7/15 completed
8/15 completed
9/15 completed
10/15 completed
11/15 completed
12/15 completed
13/15 completed
14/15 completed
15/15 completed
1/15 completed
2/15 completed
3/15 completed
4/15 completed
5/15 completed
6